# NAS for CNN

## Structural elements

- Branches
- Connections
- Layers
    - Input
        - Input shape (first input is standard, all others are calculated)
        - Batch size 
    - 2D convolutional
        - Filters
        - Kernel size
        - Padding
        - Activation
    - Pooling
        - Max
            - Size
            - Strides
            - Padding
        - Average
            - Size
            - Strides
            - Padding
    - Dropout
        - Rate
    - Batch normalisation
    - Concatenate
        - Axis
    - Flatten
    - Dense
        - Units (1, 2)
        - Activation (sigmoid, softmax)
    - Global pooling
        - Max
        - Average

## Genetic elements

- GA
    - Type
    - Number of generations
- Population
    - Size
    - Encoding type
    - Individual size
        - Variable or fixed
- Fitness
- Selection
- Reproduction
- Mutation

In [41]:
import numpy as np
import tensorflow as tf

In [30]:
population = np.array([[1,0,0],[1,0,0],[1,0,0]])

array([1, 1, 1])

In [1]:
params = {
    'input':{
        'batch':[8, 16, 32, 64, 128, 256]
        },
    'conv':{
        'kernel':['1x1', '3x3', '5x5', '7x7'],
        'filter':[2, 4, 8, 16, 32, 64, 128],
        'padding':['valid', 'same'],
        'activation':['tanh', 'relu', 'selu', 'elu']
        },
    'pool':{
        'type':['max', 'average'],
        'size':['2x2', '3x3', '4x4', '5x5'],
        'padding':['valid', 'same']
        },
    'dropout':{
        'type':['dropout','spatial2D'],
        'rate':[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
        },
    'output':{
        'type':['global', 'dense']
        }
}
    

In [5]:
params.items()


dict_items([('input', {'batch': [8, 16, 32, 64, 128, 256]}), ('conv', {'kernel': ['1x1', '3x3', '5x5', '7x7'], 'filter': [2, 4, 8, 16, 32, 64, 128], 'padding': ['valid', 'same'], 'activation': ['tanh', 'relu', 'selu', 'elu']}), ('pool', {'type': ['max', 'average'], 'size': ['2x2', '3x3', '4x4', '5x5'], 'padding': ['valid', 'same']}), ('dropout', {'type': ['dropout', 'spatial2D'], 'rate': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}), ('output', {'type': ['global', 'dense']})])

In [35]:
individual = [
    # Input layer
    'batch_size_index', 
    
    # Convolutional layer
    'kernel_index', 'filter_index', 'padding_index', 'activation_index',

    # Pooling layer
    'type_index', 'size_index', 'padding_index',

    # Dropout layer
    'type_index', 'rate_index',

    # Output layer
    'type_index',

    # Add layer
    ## Layer/Branch 1
    'indicator', 'branch_indicator', 'type', 'from', 'to',

    ## Layer/Branch 2
    'indicator', 'branch_indicator', 'type', 'from', 'to',

    ## ... variable length array

    ## Layer/Branch n
    'indicator', 'branch_indicator', 'type', 'from', 'to'


    ]
    


[8, 16, 32, 64, 128, 256]

In [104]:
def get_state_space(parameters):

    """
    Get the encoding length of an individual based on the input parameters dictionary
    """

    len_individual = 0
    len_values = []

    for layer in parameters.keys():

        for value_list in list(parameters[layer].values()):

            len_values.append(len(bin(len(value_list))) - 2) 
    

    individual_length = np.sum(np.array(len_values))

    cumulative_sum = np.cumsum(np.array(len_values))
    
    len_values = np.array(len_values)

    return individual_length, np.append(cumulative_sum - len_values, np.array([individual_length])), len_values

In [134]:
ind_len, sub_ind, len_vals = get_state_space(params)
print(ind_len, sub_ind, len_vals)

29 [ 0  3  6  9 11 14 16 19 21 23 27 29] [3 3 3 2 3 2 3 2 2 4 2]


In [57]:
def generate_population(length, n = 100, seed = None):
    """
    Generate population given the number of individuals in a population and the the required binary length
    """
    if seed != None:

        np.random.seed(seed)
        
    population = np.random.randint(0, 2, size=(n, length))
    
    return population


In [62]:
pop = generate_population(ind_len, seed=42)
print(pop.shape)

(100, 29)


In [144]:
def decoder(population, sub_indexes):

    """
    decode the bitstring vector into an integer array
    """

    decoded = []

    for individual in population:

        ind_decoded = []
        print(individual)
        for i in range(len(sub_indexes) - 1):
            
            print(sub_indexes[i])
            print(sub_indexes[i + 1])

            sub_string = ''.join(map(str, list(individual[sub_indexes[i]:sub_indexes[i + 1]])))
            print(sub_string)
            print(int(sub_string, 2))
            print('-')
            ind_decoded.append(int(sub_string, 2))

        decoded.append(ind_decoded)

    return decoded

In [145]:
dec = decoder(pop, sub_ind)
# outputs are sometimes exceding maximum allowed, e.g. dropout rate has max 9 but because we are using four bits 
# we get 16 possibilities

[0 1 0 0 0 1 0 0 0 1 0 0 0 0 1 0 1 1 1 0 1 0 1 1 1 1 1 1 1]
0
3
010
2
-
3
6
001
1
-
6
9
000
0
-
9
11
10
2
-
11
14
000
0
-
14
16
10
2
-
16
19
111
7
-
19
21
01
1
-
21
23
01
1
-
23
27
1111
15
-
27
29
11
3
-
[1 0 0 1 1 1 0 1 0 0 0 0 0 1 1 1 1 1 0 1 1 0 1 0 1 0 1 1 0]
0
3
100
4
-
3
6
111
7
-
6
9
010
2
-
9
11
00
0
-
11
14
001
1
-
14
16
11
3
-
16
19
110
6
-
19
21
11
3
-
21
23
01
1
-
23
27
0101
5
-
27
29
10
2
-
[0 0 0 0 0 0 0 1 1 0 1 1 1 1 0 1 0 1 1 1 0 1 0 1 0 1 0 0 1]
0
3
000
0
-
3
6
000
0
-
6
9
011
3
-
9
11
01
1
-
11
14
111
7
-
14
16
01
1
-
16
19
011
3
-
19
21
10
2
-
21
23
10
2
-
23
27
1010
10
-
27
29
01
1
-
[0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 1 0 1 1 0 1]
0
3
011
3
-
3
6
111
7
-
6
9
111
7
-
9
11
11
3
-
11
14
100
4
-
14
16
11
3
-
16
19
111
7
-
19
21
11
3
-
21
23
10
2
-
23
27
1011
11
-
27
29
01
1
-
[0 1 1 0 1 0 1 0 0 1 1 0 1 1 1 0 0 0 0 0 0 0 0 0 1 0 1 1 1]
0
3
011
3
-
3
6
010
2
-
6
9
100
4
-
9
11
11
3
-
11
14
011
3
-
14
16
10
2
-
16
19
000
0
-
19
21
00
0
-
21
23
00
0
-
23
27
0101

In [132]:
pop[0][0:3]

array([0, 1, 0])

In [139]:
dec[0]

[2, 1, 0, 2, 0, 2, 7, 1, 1, 15, 3]

In [26]:
model.build()

TypeError: build() missing 1 required positional argument: 'input_shape'